In [3]:
import pandas as pd
import pulp
import re

### Zielfunktion:

$$
Z = \sum_j \left( \frac{\text{LastDeadline}}{\text{Deadline}_j} \cdot \left[(\text{End}_j - \text{Arrival}_j) + \max(0, \text{End}_j - \text{Deadline}_j)\right] \right)
$$

---

In [6]:
def solve_jssp_weighted_flow_and_tardiness_by_deadline(
    df_jssp: pd.DataFrame,
    df_arrivals_deadlines: pd.DataFrame,
    solver_time_limit: int = 600,
    epsilon: float = 0.0,
    msg_print: bool = False,
    threads: int = None
) -> pd.DataFrame:
    """
    Minimiert gewichtete Summe aus Flowtime + Tardiness,
    gewichtet mit LastDeadline / Deadline_j

    Zielfunktion:
        sum_j [ (LastDeadline / Deadline_j) * ((End_j - Arrival_j) + Tardiness_j) ]
    """

    # Vorsortierung für bessere Stabilität im Solver
    df_arrivals_deadlines = df_arrivals_deadlines.sort_values("Deadline", ascending=False).reset_index(drop=True)
    df_jssp = df_jssp.sort_values(["Job", "Operation"]).reset_index(drop=True)

    # Vorbereitung: Ankunfts- und Deadlinezeiten
    arrival = df_arrivals_deadlines.set_index("Job")["Arrival"].to_dict()
    deadline = df_arrivals_deadlines.set_index("Job")["Deadline"].to_dict()
    jobs = df_arrivals_deadlines["Job"].tolist()

    last_deadline = max(deadline.values())
    weights = {job: last_deadline / deadline[job] for job in jobs}

    # Operationen pro Job extrahieren
    ops_grouped = df_jssp.groupby("Job", sort=False)
    all_ops, machines = [], set()
    for job in jobs:
        seq = []
        for _, row in ops_grouped.get_group(job).iterrows():
            op_id = row["Operation"]
            m = int(re.search(r"M(\d+)", str(row["Machine"])).group(1))
            d = float(row["Processing Time"])
            seq.append((op_id, m, d))
            machines.add(m)
        all_ops.append(seq)

    # Modell initialisieren
    n = len(jobs)
    bigM = 1e6
    prob = pulp.LpProblem("JSSP_Weighted_FlowTardiness_byDeadline", pulp.LpMinimize)

    # Entscheidungsvariablen
    starts = {(j, o): pulp.LpVariable(f"start_{j}_{o}", lowBound=0)
              for j in range(n) for o in range(len(all_ops[j]))}
    ends = {j: pulp.LpVariable(f"end_{j}", lowBound=0) for j in range(n)}
    tard = {j: pulp.LpVariable(f"tard_{j}", lowBound=0) for j in range(n)}

    # Technologische Reihenfolge und Zeitlogik
    for j, job in enumerate(jobs):
        seq = all_ops[j]
        # Erste Operation darf nicht vor Ankunft starten
        prob += starts[(j, 0)] >= arrival[job]
        # Jede Operation folgt auf die vorherige
        for o in range(1, len(seq)):
            _, _, d_prev = seq[o - 1]
            prob += starts[(j, o)] >= starts[(j, o - 1)] + d_prev
        # Endzeit des Jobs entspricht Ende der letzten Operation
        prob += ends[j] == starts[(j, len(seq) - 1)] + seq[-1][2]
        # Tardiness >= Endzeit - Deadline
        prob += tard[j] >= ends[j] - deadline[job]

    # Maschinenkonflikte: disjunkte Nutzung pro Maschine
    for m in sorted(machines):
        ops_on_m = [(j, o, all_ops[j][o][2])
                    for j in range(n)
                    for o in range(len(all_ops[j]))
                    if all_ops[j][o][1] == m]
        for i in range(len(ops_on_m)):
            j1, o1, d1 = ops_on_m[i]
            for j2, o2, d2 in ops_on_m[i + 1:]:
                if j1 == j2:
                    continue
                y = pulp.LpVariable(f"y_{j1}_{o1}_{j2}_{o2}", cat="Binary")
                prob += starts[(j1, o1)] + d1 + epsilon <= starts[(j2, o2)] + bigM * (1 - y)
                prob += starts[(j2, o2)] + d2 + epsilon <= starts[(j1, o1)] + bigM * y

    # Zielfunktion: Gewicht * (Flowtime + Tardiness)
    prob += pulp.lpSum(
        weights[jobs[j]] * ((ends[j] - arrival[jobs[j]]) + tard[j])
        for j in range(n)
    )

    # Solver starten
    solver_args = {"msg": msg_print, "timeLimit": solver_time_limit}
    if threads:
        solver_args["threads"] = threads
    prob.solve(pulp.HiGHS_CMD(**solver_args))

    # Ergebnisse extrahieren
    records = []
    for j, job in enumerate(jobs):
        for o, (op_id, m, d) in enumerate(all_ops[j]):
            st = starts[(j, o)].varValue
            ed = st + d
            records.append({
                "Job": job,
                "Operation": op_id,
                "Arrival": arrival[job],
                "Deadline": deadline[job],
                "Machine": f"M{m}",
                "Start": round(st, 2),
                "Processing Time": d,
                "End": round(ed, 2),
                "Flow time": round(ed - arrival[job], 2),
                "Tardiness": max(0.0, round(ed - deadline[job], 2)),
                "Weight": round(weights[job], 2)
            })

    df_schedule = pd.DataFrame.from_records(records)
    df_schedule = df_schedule.sort_values(["Arrival", "Start", "Job"]).reset_index(drop=True)

    print("\nSolver-Informationen:")
    print(f"  Zielfunktionswert       : {round(pulp.value(prob.objective), 4)}")
    print(f"  Solver-Status           : {pulp.LpStatus[prob.status]}")
    print(f"  Anzahl Variablen        : {len(prob.variables())}")
    print(f"  Anzahl Constraints      : {len(prob.constraints)}")

    return df_schedule



### **Gesuchte Zielfunktion:**

$$
Z = \sum_{j} \left( \frac{\text{LastDeadline}}{\text{Deadline}_j} \cdot \left[ (C_j - A_j) + \gamma \cdot \text{tard\_flag}_j \right] \right)
$$

---

### **Bedeutung der Terme:**

| Symbol                             | Bedeutung                                               |
| ---------------------------------- | ------------------------------------------------------- |
| $C_j$                              | Endzeit des Jobs $j$                                    |
| $A_j$                              | Ankunftszeit des Jobs $j$                               |
| $\text{Deadline}_j$                | Deadline des Jobs $j$                                   |
| $\text{LastDeadline}$              | späteste Deadline im gesamten System                    |
| $\text{tard\_flag}_j \in \{0, 1\}$ | 1, wenn Job $j$ verspätet ist                           |
| $\gamma$                           | feste Strafe pro verspätetem Job (z. B. $\gamma = 100$) |

---

### **Ziel:**

* **Minimierung der gewichteten Flowtime**
* **Vermeidung von Verspätungen durch diskrete Bestrafung**, je wichtiger (früher) ein Job, desto härter fällt sie ins Gewicht

In [10]:
def solve_jssp_weighted_flow_with_weighted_late_penalty(
    df_jssp: pd.DataFrame,
    df_arrivals_deadlines: pd.DataFrame,
    penalty_per_late_job: float = 100.0,
    solver_time_limit: int = 600,
    epsilon: float = 0.0,
    msg_print: bool = False,
    threads: int = None
) -> pd.DataFrame:
    """
    Minimiert gewichtete Flowtime + gewichtete Strafe für verspätete Jobs.
    Gewicht: LastDeadline / Deadline_j

    Zielfunktion:
        sum_j [ (LastDeadline / Deadline_j) * ((C_j - A_j) + gamma * tard_flag_j) ]

    """
    # Umgekehrte Pre-Sortierung nach Deadline (Jobs mit später Deadline zuerst)
    df_arrivals_deadlines = df_arrivals_deadlines.sort_values("Deadline", ascending=False).reset_index(drop=True)
    df_jssp = df_jssp.sort_values(["Job", "Operation"]).reset_index(drop=True)

    arrival = df_arrivals_deadlines.set_index("Job")["Arrival"].to_dict()
    deadline = df_arrivals_deadlines.set_index("Job")["Deadline"].to_dict()
    jobs = df_arrivals_deadlines["Job"].tolist()
    last_deadline = max(deadline.values())
    weights = {job: last_deadline / deadline[job] for job in jobs}

    ops_grouped = df_jssp.groupby("Job", sort=False)
    all_ops, machines = [], set()
    for job in jobs:
        seq = []
        for _, row in ops_grouped.get_group(job).iterrows():
            op_id = row["Operation"]
            m = int(re.search(r"M(\d+)", str(row["Machine"])).group(1))
            d = float(row["Processing Time"])
            seq.append((op_id, m, d))
            machines.add(m)
        all_ops.append(seq)

    n = len(jobs)
    bigM = 1e6
    prob = pulp.LpProblem("JSSP_WeightedFlow_LatePenalty", pulp.LpMinimize)

    starts = {(j, o): pulp.LpVariable(f"start_{j}_{o}", lowBound=0)
              for j in range(n) for o in range(len(all_ops[j]))}
    ends = {j: pulp.LpVariable(f"end_{j}", lowBound=0) for j in range(n)}
    tard_flag = {j: pulp.LpVariable(f"tard_flag_{j}", cat="Binary") for j in range(n)}

    for j, job in enumerate(jobs):
        seq = all_ops[j]
        prob += starts[(j, 0)] >= arrival[job]
        for o in range(1, len(seq)):
            _, _, d_prev = seq[o - 1]
            prob += starts[(j, o)] >= starts[(j, o - 1)] + d_prev
        prob += ends[j] == starts[(j, len(seq) - 1)] + seq[-1][2]
        prob += ends[j] <= deadline[job] + bigM * tard_flag[j]

    for m in sorted(machines):
        ops_on_m = [(j, o, all_ops[j][o][2])
                    for j in range(n)
                    for o in range(len(all_ops[j]))
                    if all_ops[j][o][1] == m]
        for i in range(len(ops_on_m)):
            j1, o1, d1 = ops_on_m[i]
            for j2, o2, d2 in ops_on_m[i + 1:]:
                if j1 == j2:
                    continue
                y = pulp.LpVariable(f"y_{j1}_{o1}_{j2}_{o2}", cat="Binary")
                prob += starts[(j1, o1)] + d1 + epsilon <= starts[(j2, o2)] + bigM * (1 - y)
                prob += starts[(j2, o2)] + d2 + epsilon <= starts[(j1, o1)] + bigM * y

    # Zielfunktion
    prob += pulp.lpSum(
        weights[jobs[j]] * ((ends[j] - arrival[jobs[j]]) + penalty_per_late_job * tard_flag[j])
        for j in range(n)
    )

    solver_args = {"msg": msg_print, "timeLimit": solver_time_limit}
    if threads:
        solver_args["threads"] = threads
    prob.solve(pulp.HiGHS_CMD(**solver_args))

    records = []
    for j, job in enumerate(jobs):
        for o, (op_id, m, d) in enumerate(all_ops[j]):
            st = starts[(j, o)].varValue
            ed = st + d
            records.append({
                "Job": job,
                "Operation": op_id,
                "Arrival": arrival[job],
                "Deadline": deadline[job],
                "Machine": f"M{m}",
                "Start": round(st, 2),
                "Processing Time": d,
                "End": round(ed, 2),
                "Flow time": round(ed - arrival[job], 2),
                "Tardiness flag": int(tard_flag[j].varValue),
                "Weight": round(weights[job], 2)
            })

    df_schedule = pd.DataFrame.from_records(records)
    df_schedule = df_schedule.sort_values(["Arrival", "Start", "Job"]).reset_index(drop=True)

    print("\nSolver-Informationen:")
    print(f"  Zielfunktionswert       : {round(pulp.value(prob.objective), 4)}")
    print(f"  Solver-Status           : {pulp.LpStatus[prob.status]}")
    print(f"  Anzahl Variablen        : {len(prob.variables())}")
    print(f"  Anzahl Constraints      : {len(prob.constraints)}")

    return df_schedule


In [ ]:
"""

df_plan = solve_jssp_weighted_tardiness_with_flags(
    df_jssp=df_jssp_curr,
    df_arrivals_deadlines=df_times_curr,
    alpha=100,                # Gewicht für verspätete Jobs (binär)
    beta=1.0,                  # Gewicht für echte Tardiness
    solver_time_limit=1800,
    threads=4,
    msg_print=False
)

print(f"   Dauer {(time.time() - info_time):.4f} Sekunden.")
df_plan

"""